In [2]:
from fastNLP import DataSet
import string
import torch
import pickle
import numpy as np

/home/lyhe/anaconda3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [27]:
def nli_preprocessing(path):
    ds = DataSet.read_csv(path, sep='\t')
    punct_table = str.maketrans({key: ' ' for key in string.punctuation})
    ds.apply(lambda x: x['sentence1'].lower().translate(punct_table).strip(), new_field_name='premise')
    ds.apply(lambda x: x['sentence2'].lower().translate(punct_table).strip(), new_field_name='hypothesis')
    ds.drop(lambda x: x['gold_label'] == '-')
    def labeling(ins):
        label_str = ins['gold_label'].strip()
        if label_str == 'entailment':
            return 0
        elif label_str == 'neutral':
            return 1
        elif label_str == 'contradiction':
            return 2
        else:
            return -1
    ds.apply(labeling, new_field_name='label', is_target=True)
    ds.apply(lambda x: x['premise'].split(), new_field_name='premise_words')
    ds.apply(lambda x: x['hypothesis'].split(), new_field_name='hypothesis_words')
    delete_labels = ['sentence1_binary_parse','sentence2_binary_parse',
                               'sentence1_parse','sentence2_parse','sentence1', 'sentence2', 'captionID','pairID',
                               'label1','label2','label3','label4','label5']
    for name in delete_labels:
        ds.delete_field(name)
    return ds

In [3]:
train_path = '/home/lyhe/Fudan-Deep_Learning/ESIM/snli_data/snli_1.0/snli_1.0_train.txt'
ds_train = nli_preprocessing(train_path)

In [10]:
dev_path = '/home/lyhe/Fudan-Deep_Learning/ESIM/snli_data/snli_1.0/snli_1.0_dev.txt'
ds_dev = nli_preprocessing(train_path)

In [11]:
test_path = '/home/lyhe/Fudan-Deep_Learning/ESIM/snli_data/snli_1.0/snli_1.0_test.txt'
ds_test = nli_preprocessing(train_path)

In [4]:
from fastNLP import Vocabulary

In [ ]:
vocab = Vocabulary(min_freq=1)
ds_train.apply(lambda x: [vocab.add(word) for word in x['premise_words']+x['hypothesis_words']])
print(len(vocab))
with open('vocab.pkl', 'wb') as file:
     pickle.dump(vocab,file)

In [5]:
with open('vocab.pkl', 'rb') as file:
     vocab = pickle.load(file)

In [6]:
from fastNLP.io.embed_loader import EmbedLoader

In [7]:
# emb_dim = 300
# emb_file = '/home/lyhe/Fudan-Deep_Learning/ESIM/ESIM-master/data/embeddings/glove.840B.300d.txt'
# emb_file = '/home/lyhe/Fudan-Deep_Learning/ESIM/ESIM-master/data/embeddings/mini_glove.txt'
# emb_matrix = EmbedLoader().fast_load_embedding(emb_dim=emb_dim, emb_file=emb_file, vocab=vocab)
# with open('emb_matrix.pkl', 'wb') as file:
#     pickle.dump(emb_matrix, file)
with open('emb_matrix.pkl', 'rb') as file:
    emb_matrix= pickle.load(file)

In [13]:
ds_train.apply(lambda x: [vocab.to_index(word) for word in x['premise_words']], 
                 new_field_name='p_seq', is_input=True)
ds_train.apply(lambda x: [vocab.to_index(word) for word in x['hypothesis_words']], 
                 new_field_name='h_seq', is_input=True)
ds_dev.apply(lambda x: [vocab.to_index(word) for word in x['premise_words']], 
                 new_field_name='p_seq', is_input=True)
ds_dev.apply(lambda x: [vocab.to_index(word) for word in x['hypothesis_words']], 
                 new_field_name='h_seq', is_input=True)
ds_test.apply(lambda x: [vocab.to_index(word) for word in x['premise_words']], 
                 new_field_name='p_seq', is_input=True)
ds_test.apply(lambda x: [vocab.to_index(word) for word in x['hypothesis_words']], 
                 new_field_name='h_seq', is_input=True)

In [ ]:
with open('ds_train_new.pkl', 'wb') as file:
    pickle.dump(ds_train, file)
with open('ds_dev_new.pkl', 'wb') as file:
    pickle.dump(ds_dev, file)
with open('ds_test_new.pkl', 'wb') as file:
    pickle.dump(ds_test, file)

In [8]:
with open('ds_train_new.pkl', 'rb') as file:
    ds_train = pickle.load(file)
with open('ds_dev_new.pkl', 'rb') as file:
    ds_dev = pickle.load(file)
with open('ds_test_new.pkl', 'rb') as file:
    ds_test = pickle.load(file)

In [11]:
import model

In [12]:
mymodel = model.ESIM(hidden_size = 300, embeds_dim = 300, linear_size = 300, num_word = len(vocab))
mymodel.load_pretrained_glove(emb_matrix)

In [13]:
from fastNLP import CrossEntropyLoss
from fastNLP import AccuracyMetric
from fastNLP import Trainer
from fastNLP.core.optimizer import Adam

In [43]:
trainer = Trainer(model=mymodel,
                  train_data=ds_train,
                  dev_data=ds_dev,
                  loss=CrossEntropyLoss(pred="pred", target="label"),
                  metrics=AccuracyMetric(),
                  optimizer=Adam(lr=0.0004, weight_decay=0), 
                  n_epochs=64,
                  use_cuda=True)
trainer.train()

training epochs started 2019-01-17 00-34-17


Epoch 1/64. Step:17168/1098752. AccuracyMetric: acc=0.90192
Epoch 2/64. Step:34336/1098752. AccuracyMetric: acc=0.90263
Epoch 3/64. Step:51504/1098752. AccuracyMetric: acc=0.902633
Epoch 4/64. Step:68672/1098752. AccuracyMetric: acc=0.901969
Epoch 5/64. Step:85840/1098752. AccuracyMetric: acc=0.903058
Epoch 6/64. Step:103008/1098752. AccuracyMetric: acc=0.899621
Epoch 7/64. Step:120176/1098752. AccuracyMetric: acc=0.903056
Epoch 8/64. Step:137344/1098752. AccuracyMetric: acc=0.902937
Epoch 9/64. Step:154512/1098752. AccuracyMetric: acc=0.901412
Epoch 10/64. Step:171680/1098752. AccuracyMetric: acc=0.902892
Epoch 11/64. Step:188848/1098752. AccuracyMetric: acc=0.903702
Epoch 12/64. Step:206016/1098752. AccuracyMetric: acc=0.902919
Epoch 13/64. Step:223184/1098752. AccuracyMetric: acc=0.903507
Epoch 14/64. Step:240352/1098752. AccuracyMetric: acc=0.90356
Epoch 15/64. Step:257520/1098752. AccuracyMetric: acc=0.905406
Epoch 16/64. Step:274688/1098752. AccuracyMetric: acc=0.904
Epoch 17/64.

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Epoch 19/64. Step:326192/1098752. AccuracyMetric: acc=0.904956


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Epoch 54/64. Step:927072/1098752. AccuracyMetric: acc=0.90346


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Epoch 55/64. Step:944240/1098752. AccuracyMetric: acc=0.902916
Epoch 56/64. Step:961408/1098752. AccuracyMetric: acc=0.9052


KeyboardInterrupt: 

In [42]:
#we don't have enough time, so we interrup the training
from fastNLP import Tester

tester = Tester(data=ds_test, model=mymodel, metrics=AccuracyMetric())
acc = tester.test()

[tester] 
AccuracyMetric: acc=0.901354


In [44]:
torch.save(mymodel, 'snli_model.pkl')